# Checkpoint Verification v2

### Checkpoint Verification Summary

This configuration reflects corrected checkpoint availability based on findings from  
`verify_setup.ipynb`.

**What’s been verified:**
- Uses the HuggingFace `revision` parameter  
  *(instead of the deprecated `checkpoint_value`)*
- Tests **8 verified checkpoints** rather than the original 10
- Confirms successful loading through the  
  **HuggingFace → HookedTransformer** pipeline

In [ ]:
pip install transformers==4.57.6 transformer_lens==2.17.0 accelerate==1.12.0 huggingface_hub==0.36.1

In [ ]:
import transformers, huggingface_hub, accelerate
assert transformers.__version__ == "4.57.6", f"RESTART KERNEL! Got transformers {transformers.__version__}"
assert huggingface_hub.__version__ == "0.36.1", f"RESTART KERNEL! Got huggingface_hub {huggingface_hub.__version__}"
print(f"✅ transformers={transformers.__version__}, huggingface_hub={huggingface_hub.__version__}, accelerate={accelerate.__version__}")

In [ ]:
import sys
import time

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Corrected checkpoint map (8 verified checkpoints)
CHECKPOINT_STEPS = [
    "step0",
    "step15000",
    "step30000",
    "step60000",
    "step90000",
    "step120000",
    "step140000",
    "step143000",  # Final checkpoint (proxy for "late training")
]

MODEL_SIZES = ["160m", "1b", "2.8b"]

# Quick availability check: test 3 representative checkpoints per model
SPOT_CHECK_STEPS = ["step0", "step30000", "step143000"]

## Step 1: Spot-Check Checkpoint Availability (HF Download Only)

Test **early**, **mid**, and **late** checkpoints across all **three model sizes** using  
`AutoModelForCausalLM` with the HuggingFace `revision` parameter.

In [ ]:
results = {}

for size in MODEL_SIZES:
    for step in SPOT_CHECK_STEPS:
        model_name = f"EleutherAI/pythia-{size}-deduped"
        key = f"{size}/{step}"
        try:
            start = time.time()
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                revision=step,
                dtype=torch.float32,        # was: torch_dtype (deprecated in 5.1.0)
                device_map="auto",
                use_safetensors=False,       # suppress auto-conversion thread errors
            )
            elapsed = time.time() - start
            param_count = sum(p.numel() for p in model.parameters()) / 1e6
            results[key] = {"status": "OK", "time_s": elapsed, "params_M": param_count}
            print(f"✅ {key}: {elapsed:.1f}s, {param_count:.0f}M params")
            del model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        except Exception as e:
            results[key] = {"status": "FAIL", "error": str(e)}
            print(f"❌ {key}: {e}")

## Step 2: Verify All 8 Checkpoints (160M Model)

Run a full sweep across all **8 corrected checkpoints** using the **smallest (160M)** model  
to confirm availability before scaling up.

In [ ]:
model_name_160m = "EleutherAI/pythia-160m-deduped"
all_ok = True

for step in CHECKPOINT_STEPS:
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_name_160m,
            revision=step,
            dtype=torch.float32,        # was: torch_dtype
            use_safetensors=False,       # suppress auto-conversion thread errors
        )
        print(f"✅ 160m {step}: OK")
        del model
    except Exception as e:
        all_ok = False
        print(f"❌ 160m {step}: {e}")

print(f"\nAll 8 checkpoints available: {'YES' if all_ok else 'NO'}")

## Step 3: End-to-End HookedTransformer Loading Test

Verify the full pipeline from **HuggingFace model loading** to  
**HookedTransformer initialization** using the `revision` parameter,  
implemented via `src/utils_model.py`.

In [ ]:
sys.path.insert(0, "..")
from src.utils_model import load_pythia_with_checkpoint

# Test the full pipeline on 160m at step30000
start = time.time()
hooked_model = load_pythia_with_checkpoint("160m", "step30000")
elapsed = time.time() - start
print(f"✅ HookedTransformer loaded in {elapsed:.1f}s")
print(f"   Model: {hooked_model.cfg.model_name}")
print(f"   Layers: {hooked_model.cfg.n_layers}")
print(f"   Device: {hooked_model.cfg.device}")

## Step 4: Attention Extraction Sanity Check

Confirm that **attention cache extraction** works correctly with models  
loaded via the `revision` parameter.

In [ ]:
test_input = "A screen reader is"
tokens = hooked_model.to_tokens(test_input)

start = time.time()
logits, cache = hooked_model.run_with_cache(tokens)
cache_time = time.time() - start

cache_bytes = sum(v.element_size() * v.nelement() for v in cache.values())
print(f"✅ Attention cache extracted in {cache_time:.2f}s")
print(f"   Cache memory: {cache_bytes / 1e6:.1f} MB")
print(f"   Token count: {tokens.shape[1]}")
print(f"   Logits shape: {logits.shape}")

# Verify attention pattern shape
attn_key = [k for k in cache.keys() if "attn" in k and "pattern" in k][0]
print(f"   Attention pattern shape: {cache[attn_key].shape}")

del hooked_model, logits, cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Summary

Run all cells above. Expected outcomes are listed below.

| Check | Expected Result |
|------:|-----------------|
| Spot-check (3 sizes × 3 steps) | All 9 combinations pass ✅ |
| Full 8-checkpoint sweep (160M) | All 8 checkpoints pass ✅ |
| HookedTransformer pipeline | Loads successfully via `utils_model.py` ✅ |
| Attention extraction | Cache extracted; attention pattern shapes correct ✅ |

**If all checks pass → proceed to _Stage 1A (Behavioral Probes)_:**  
**8 checkpoints × 3 models × 3 terms = 72 total combinations.**